### Libraries

In [1]:
import requests 
import json
from dotenv import load_dotenv
import os
import pandas as pd

from functools import reduce
import operator
import geopandas as gdp
import shapely


from pymongo import MongoClient

In [2]:
load_dotenv()

True

### Creando los geocodes

En este caso, voy a partir de tres ubicaciones diferentes:

- España **-->** Madrid
- Paises Bajos **-->** Amsterdam
- Francia **-->** Paris 
- Canadá **-->** Montréal 

In [3]:
countries = ["madrid","paris", "ottawa"]

EN ESTA FUNCIOÓN DE AQUÍ TENDRÍA QUE MIRAR PARA METER EL TRY AND EXCEPT

In [4]:
def get_coordinates(lista):
    d = {}
    try: 
        for country in countries:
            locations = requests.get(f'https://geocode.xyz/{country}?json=1')
            locations = locations.json()
            d[country] = locations
    except:
        return "Error getting data"
        
    return d     

In [5]:
c = get_coordinates(countries)

In [6]:
c['madrid']
for k, v in c.items():
    print(v.keys())

dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])


In [7]:
def exctract_coordinates(dictionary):
    dict_empty = {}
    for k, v in c.items():
        coor = [float(v["longt"]),float(v["latt"])]
        dict_empty[k] = coor
    return dict_empty

In [8]:
coordinates = exctract_coordinates(c)

In [9]:
coordinates

{'madrid': [-3.6793, 40.42955],
 'paris': [2.3428, 48.85756],
 'ottawa': [-75.70531, 45.35917]}

### Llamadas a la API

meter un parametro de radio = metros desde la ubicación dada
meter el parametro de limit para que no me pete, tiene un limite a la hora

In [10]:
token1= os.getenv("token1")
token2 = os.getenv("token2")

In [11]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

#### Llamadas a la API

#####  Codes to queries

Dentro de todas las queries hay algunas que tienen que estar muy cerca o otras que da igual que estén un pco más lejos, lo que haré separarlas por grupos

In [12]:
school = '52e81612bcbc57f1066b7a45'
vegan = "4bf58dd8d48988d1d3941735"

queries = ["Preschool", "Vegan Restaurant"]


##### Setting the parameters

In [14]:
def datos_madrid (*args):
    '''
    descripcion de lo que hace la función aquí
    '''  
    d = {}
    
    for i in args: 
        parametros = {"client_id" : token1,
                  "client_secret" : token2,
                  "v": "20180323",
                  "ll": f"{coordinates['madrid'][1]},{coordinates['madrid'][0]}", ## esto lo puedo hacer con kwargs
                  "query":i,
                  "limit": 100}  

        resp = requests.get(url= url_query, params=parametros)
        data = json.loads(resp.text)
        d[i] = data
    return d

In [15]:
results = datos_madrid(*["Preschool", "Vegan Restaurant"])

In [16]:
results.keys()#salen como resultado 7 diccionarios de cada una de las queries que he pedido. Despues de esto habría que 
#separar cada uno de los diccionarios y proceder a su limpieza

dict_keys(['Preschool', 'Vegan Restaurant'])

**Para separar los distintos diccionarios que he genarado con la función anterior:**

In [110]:
def create_separate_dictionaries(result):
    for key, values in result.items():
        if key == "Preschool":
            school = result.get(key)

        else:
            vegan = result.get(key)
            
    return school, vegan

In [111]:
school, vegan = create_separate_dictionaries(results)

In [123]:
args = [school, vegan]
type(all_results)


list

### Limpieza de los resultados


In [168]:
def cleaning_dictionaries(*args):
    school = args[0]
    for key, values in school.items():

        response = school.get('response')
        decoded = response.get('groups')[0]
        final_school = decoded.get('items')

    vegan = args[1]
    for key, values in vegan.items():

        response = vegan.get('response')
        decoded = response.get('groups')[0]
        final_vegan = decoded.get('items')

    return final_school, final_vegan

'''
print(key, "-->", value, '/n')
'''

'\nprint(key, "-->", value, \'/n\')\n'

In [171]:
school_results, vegan_results = cleaning_dictionaries(*args)               

### Creating dataframes

In [ ]:
mapa_nombre = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [ ]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

####  For school

In [ ]:
data_schools = []
for dic in school_data:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_schools.append(paralista)

In [ ]:
data_schools[0]

####  For vegan

In [ ]:
data_vegan = []
for dic in vegan_data:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_vegan.append(paralista)
data_vegan

###  Saving dataframes

#### School query

In [ ]:
df_pets = pd.DataFrame(data_schools)
df_pets.to_csv("school.csv", index = False)
df_pets.head()

#### Vegan query

In [ ]:
df_gym = pd.DataFrame(data_vegan)
df_gym.to_csv("vegan.csv", index = False)
df_gym.head()

In [ ]:
def create_separate_dictionaries (dictonary_queries):
    filterByKey = lambda keys: {x: results[x] for x in keys}
    for key, values in dictonary_queries.items():
        
        return filterByKey([values])
    
    results *args